In [1]:
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Bring biomass data into dataframe

In [2]:
Observations = pd.read_excel('C:\Users\cflhxb\Dropbox\APSIMOats\Observations.xlsx',sheetname='Observations')
obsPhenology = pd.read_excel('C:\Users\cflhxb\Dropbox\APSIMOats\Observations.xlsx',sheetname='obsPhenology')
ObsStructure = pd.read_excel('C:\Users\cflhxb\Dropbox\APSIMOats\Observations.xlsx',sheetname='ObsStructure')
Obs2016 = pd.read_excel('C:\Users\cflhxb\Dropbox\APSIMOats\Observations.xlsx',sheetname='Obs2016')
Obs2015 = pd.read_excel('C:\Users\cflhxb\Dropbox\APSIMOats\Observations.xlsx',sheetname='Obs2015')
RS_SW_Obs = pd.read_excel('C:\Users\cflhxb\Dropbox\APSIMOats\Observations.xlsx',sheetname='RS_SW_Obs')
RS_Oats_9899_leaf = pd.read_excel('C:\Users\cflhxb\Dropbox\APSIMOats\Observations.xlsx',sheetname='RS_Oats_9899_leaf')
RS_Oats_9899_deadLeaf = pd.read_excel('C:\Users\cflhxb\Dropbox\APSIMOats\Observations.xlsx',sheetname='RS_Oats_9899_deadLeaf')
RS_Oats_Cover = pd.read_excel('C:\Users\cflhxb\Dropbox\APSIMOats\Observations.xlsx',sheetname='RS_Cover_obs')

In [3]:
AllDataTabs = [Observations,obsPhenology,ObsStructure,Obs2016,Obs2015,RS_SW_Obs,RS_Oats_9899_leaf,RS_Oats_9899_deadLeaf,RS_Oats_Cover]

## Make a column index

In [4]:
ColumnIndex = Observations.columns
for Table in AllDataTabs:
    ColumnIndex = ColumnIndex  | Table.columns  # the union of the merged data frame and the next in the list

## Make row index

In [5]:
RowIndexData = Observations.ix[:,Observations.columns.isin(['SimulationName','Clock.Today'])]
for Table in AllDataTabs:
    RowIndexData = pd.concat([RowIndexData,Table.ix[:,Table.columns.isin(['SimulationName','Clock.Today'])]],axis=0)
RowIndex = RowIndexData.drop_duplicates()
RowIndex.set_index(['SimulationName','Clock.Today'],inplace=True)

## Make empty Data frame

In [6]:
ObsDataFrame = pd.DataFrame(index=RowIndex.index, columns=ColumnIndex)

## Set indexes in the smaller tables and deal to duplicate rows

In [7]:
from collections import Counter
def MakeMean(ser):
    Series = pd.Series(index=ser.columns)
    for Group in ser.columns:
        if isinstance(Group, float):
            Series[Group] = ser[Group].mean()
        else:
            Series[Group] = Counter(ser[Group]).most_common(1)[0][0]
    return Series        

In [8]:
CompressedData = []
for Table in AllDataTabs:
    Table.set_index(['SimulationName','Clock.Today'],inplace=True)
    Compressed = Table.groupby(level=['SimulationName','Clock.Today']).apply(MakeMean)
    CompressedData.append(Compressed)

In [9]:
for DataTable in range(len(CompressedData)):
    ObsDataFrame.update(CompressedData[DataTable])


In [10]:
ObsDataFrame.to_excel('C:\Users\cflhxb\Dropbox\Observed.xlsx',merge_cells=False,sheet_name='Observed')